# Create Datasets

This notebook contains information on downloading and saving data ready for zipline ingest.

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from random import randint
from time import sleep
from pathlib import Path
import requests
import numpy as np
import pandas as pd
import time
from time import time
from datetime import datetime
from pandas import Timestamp
import datetime as dt
from tqdm import tqdm
from datetime import datetime, timedelta, date
from pandas.tseries.offsets import BDay
from yahoofinancials import YahooFinancials

In [3]:
idx = pd.IndexSlice

In [4]:
def fill_missing_values(df_data):
    df_data.fillna(method='ffill', inplace=True)
    df_data.fillna(method='bfill', inplace=True)

In [5]:
# start and end dates 
end = date.today() - pd.tseries.offsets.BusinessDay(n = 1)
start = end - 20 * 252 * pd.tseries.offsets.BDay()

In [6]:
end.strftime("%Y-%m-%d")

'2023-02-28'

In [7]:
start.strftime("%Y-%m-%d")

'2003-11-04'

## Prep Data for Zipline Ingest

we prepare the csv data for zipline ingest later

In [8]:
def download_csv_data(symbol, start_date, end_date, freq, path):
    yf = YahooFinancials(symbol)
    res = yf.get_historical_price_data(str(start_date), str(end_date),freq)
    if not res or symbol not in res or 'prices' not in res[symbol]:
        ValueError('Fetching price data for "{}" failed.'.format(symbol))
    prices = res[symbol]['prices']
    df = pd.DataFrame({'open': [p['open'] for p in prices],
                       'high': [p['high'] for p in prices],
                       'low': [p['low'] for p in prices],
                       'close': [p['close'] for p in prices],
                       'volume': [p['volume'] for p in prices],}, index=[pd.Timestamp(d['formatted_date']) for d in prices])
    if 'dividend' in prices:
        df['dividend'] = [p['dividend'] for p in prices]
    else:
        df['dividend'] = 0

    if 'split' in prices:
        df['split'] = [p['split'] for p in prices]
    else:
        df['split'] = 1
    df.sort_index(inplace=True)
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize('UTC').normalize()
    df.index = df.index.strftime("%Y-%m-%d")
    df.index.name='date'
    df['volume'] = df['volume'].astype(np.float64)
    df['dividend'] =  df['dividend'].astype(np.float64)
    df['split'] =  df['split'].astype(np.float64)
    df.to_csv(path, header=True, index=True)
    delay = randint(1,5) # random delay 1 to 5 sec 
    sleep(delay)


### Set Data Path

Modify path if you would like to store the data elsewhere and change the notebooks accordingly


In [9]:
results_path = Path('data')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [10]:
# set the csv path 
csv_path = Path('data/csv')
if not csv_path.exists():
    csv_path.mkdir(parents=True)

# Nasdaq 100

In [11]:
ndx_path_tmp = Path('data/csv/ndx/tmp')
ndx_path = Path('data/csv/ndx/daily')
if not ndx_path_tmp.exists() and not ndx_path.exists():
    ndx_path_tmp.mkdir(parents=True)
    ndx_path.mkdir(parents=True)

#### Download html table with Nasdaq 100 constituents

In [12]:
ndx_url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
nasdaq_constituents = pd.read_html(ndx_url, header=0)[4].rename(columns=str.lower)

In [13]:
nasdaq_constituents.head()

,company,ticker,gics sector,gics sub-industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe Inc.,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align Technology,ALGN,Health Care,Health Care Supplies


In [14]:
nasdaq_constituents.columns = ['company', 'ticker', 'sector', 'industry']
nasdaq_constituents.to_csv('data/ndx_info.csv')

In [15]:
assets = nasdaq_constituents.ticker

In [16]:
assets

0      ATVI
1      ADBE
2       ADP
3      ABNB
4      ALGN
       ... 
96      WBD
97     WDAY
98      XEL
99       ZM
100      ZS
Name: ticker, Length: 101, dtype: object

In [17]:
len(assets)

101

### Get Data for Nasdaq Assets

#### Download Prices 

In [18]:
for ass in tqdm(assets):
    try:
        download_csv_data(symbol=ass, start_date=start.strftime("%Y-%m-%d"), end_date=end.strftime("%Y-%m-%d"), freq='daily', path='data/csv/ndx/tmp/'+ass+'.csv')
    except Exception as e:
        print(e)

100%|██████████| 101/101 [05:46<00:00,  3.43s/it]


for tick in tqdm(assets):
    try:
        path ='data/csv/ndx/tmp/'+tick+'.csv'
        d = pd.read_csv(path, index_col=[0])
        d.index = pd.DatetimeIndex(d.index)
        d.index = d.index.strftime("%Y-%m-%d")
        d.to_csv(path, header=True, index=True)
    except:
        pass

Load data 

In [19]:
out = []
for tick in tqdm(assets):
    try:
        path='data/csv/ndx/tmp/'+tick+'.csv'
        d=pd.read_csv(path, index_col=[0])
        d.index.name='date'
        d=d.reset_index()
        cols = [(tick, col) for col in d.columns]
        d['ticker'] = tick
        d.set_index(['date', 'ticker'], inplace=True)
        out.append(d)
    except:
        pass    
    
df_adj = pd.concat(out)

100%|██████████| 101/101 [00:00<00:00, 159.39it/s]


In [20]:
df_adj

,,open,high,low,close,volume,dividend,split
date,ticker,,,,,,,
2003-11-04,ATVI,2.859375,2.906250,2.812500,2.872500,5360000.0,0.0,1.0
2003-11-05,ATVI,2.814375,2.915625,2.810625,2.836875,6768000.0,0.0,1.0
2003-11-06,ATVI,2.848125,2.870625,2.791875,2.805000,14337600.0,0.0,1.0
2003-11-07,ATVI,2.857500,2.994375,2.775000,2.919375,19839467.0,0.0,1.0
2003-11-10,ATVI,2.872500,2.936250,2.829375,2.915625,10602667.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2023-02-21,ZS,130.210007,131.350006,127.342003,127.839996,1502200.0,0.0,1.0
2023-02-22,ZS,131.100006,133.449997,129.600006,132.160004,1886000.0,0.0,1.0
2023-02-23,ZS,134.000000,134.425003,130.710007,133.500000,1437000.0,0.0,1.0


### Remove outliers

In [21]:
df = df_adj.close.unstack('ticker')
pmax = df.pct_change().max()
pmin = df.pct_change().min()
to_drop = pmax[pmax > 1].index.union(pmin[pmin<-1].index)
len(to_drop)

0

In [22]:
df = df_adj.drop(to_drop, level='ticker')

In [23]:
df.to_csv('data/ndx_data.csv')

In [24]:
ndx_tickers = pd.DataFrame(df.index.unique('ticker'))

In [25]:
ndx_tickers.to_csv("data/tickers/ndx_tickers.csv")

In [26]:
ndx_tickers = list(ndx_tickers.ticker)

In [27]:
# move data into final folder
for tick in tqdm(ndx_tickers):
    path_tmp = 'data/csv/ndx/tmp/'+tick+'.csv'
    path = 'data/csv/ndx/daily/'+tick+'.csv'
    df = pd.read_csv(path_tmp, index_col=[0])
    fill_missing_values(df)
    df.to_csv(path, header=True, index=True)

100%|██████████| 101/101 [00:02<00:00, 48.20it/s]
